# Final

non_nan_values=train[feat].dropna().values #only continuous + integer --> label for categorical
train[feat]= train. apply( lambda x: np.random.choice(non_nan_values) if pd.isna(x) else x)





In [15]:
import sklearn
print(sklearn.__version__) # need to down grade for missing forest

1.5.2


In [16]:
# # !conda install scikit-learn=0.22.2
# !conda install -c conda-forge python=3.6.5
# !conda install -c conda-forge scikit-learn=0.22.2

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

# from missingpy import MissForest 대신에 아래 사용

# missing 1 : Iterative imputer 사용 - 복잡, overfit 위험, but 소규모 + feature간 연결관계 강할경우 good
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# imputer = IterativeImputer()
# X_imputed = imputer.fit_transform(X)

#missing 2: simple imputer 사용, mean과 mode(많은수) 로 대체..
from sklearn.impute import SimpleImputer


from sklearn.model_selection import cross_val_score

# !conda install xgboost
import xgboost as xgb




print(sklearn.__version__)

1.5.2


In [44]:
# Load your dataset
train_data = pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")


In [50]:
train_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,3157.0,NaN,24472.0,F,NaN,NaN,NaN,N,1.9,NaN,3.81,NaN,NaN,NaN,NaN,141.0,10.9,4.0,C
1,1,1568.0,Placebo,19698.0,F,Y,Y,N,Y,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,12.7,4.0,D
2,2,1367.0,NaN,20819.0,F,NaN,NaN,NaN,N,2.0,NaN,3.05,NaN,NaN,NaN,NaN,80.0,11.3,4.0,D
3,3,1092.0,NaN,14610.0,F,NaN,NaN,NaN,N,2.9,NaN,3.73,NaN,NaN,NaN,NaN,337.0,10.2,4.0,C
4,4,1980.0,NaN,18628.0,F,NaN,NaN,NaN,N,0.5,NaN,3.12,NaN,NaN,NaN,NaN,190.0,11.2,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,1328.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.7,NaN,3.14,NaN,NaN,NaN,NaN,425.0,10.9,3.0,D
14996,14996,904.0,D-penicillamine,22336.0,F,N,N,Y,N,0.6,396.0,3.53,102.0,1257.0,137.95,118.0,216.0,10.6,1.0,D
14997,14997,989.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.6,NaN,3.40,NaN,NaN,NaN,NaN,388.0,11.0,3.0,C
14998,14998,790.0,Placebo,19994.0,F,N,Y,N,N,3.2,NaN,3.41,86.0,1790.0,134.85,NaN,149.0,11.0,4.0,D


In [89]:
train_label=train_data['Status']
train_label

0        C
1        D
2        D
3        C
4        C
        ..
14995    D
14996    D
14997    C
14998    D
14999    C
Name: Status, Length: 15000, dtype: object

In [81]:
import pandas as pd
import numpy as np

def engineer_liver_cirrhosis_features_row_wise(df):
    clinical_signs = ['Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
    all_features = ['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Bilirubin']

    def process_row(row):
        # 1. Count NA values in clinical signs
        row['Clinical_Signs_NA_Count'] = sum(pd.isna(row[sign]) for sign in clinical_signs)
        row['All_Clinical_Signs_NA'] = int(row['Clinical_Signs_NA_Count'] == 4)
        
#         # 2. Bilirubin and Edema relationship
#         row['Bilirubin_Edema_Both_NA'] = int(pd.isna(row['Bilirubin']) and pd.isna(row['Edema']))
        
#         # 3. Age-related features
#         row['Age_NA_Impact'] = sum(pd.isna(row[feat]) for feat in all_features) if pd.isna(row['Age']) else 0
        
        # 4. Drug and clinical features relationship
        row['Drug_NA_Clinical_NA_Count'] = sum(pd.isna(row[feat]) for feat in clinical_signs) if pd.isna(row['Drug']) else 0
        
#         # 5. N_Days relationship with other features
#         row['N_Days_NA_Impact'] = sum(pd.isna(row[feat]) for feat in all_features) if pd.isna(row['N_Days']) else 0
        
#         # 6. Clinical Severity Score (consider N/A as 0, non-numeric as 0)
#         row['Clinical_Severity_Score'] = sum(float(row[sign]) if pd.notna(row[sign]) and str(row[sign]).replace('.', '', 1).isdigit() else 0 for sign in clinical_signs)
        
        # 7. Bilirubin status
        if pd.isna(row['Bilirubin']):
            row['Bilirubin_Status'] = 'NA'
        elif row['Bilirubin'] > df['Bilirubin'].quantile(0.95):
            row['Bilirubin_Status'] = 'High'
        else:
            row['Bilirubin_Status'] = 'Normal'
        
        # 8. Clinical signs interaction with sex
        for sign in clinical_signs:
            row[f'{sign}_Sex_Interaction'] = int(pd.notna(row[sign]) and row[sign] and row['Sex'] == 'M')
        
        # 9. NA pattern encoding
        row['NA_Pattern'] = ''.join(['1' if pd.isna(row[feat]) else '0' for feat in clinical_signs + ['Edema', 'Bilirubin']])
        
        return row

    # Apply the function row-wise
    df = df.apply(process_row, axis=1)
    
    return df

# Example usage:
# df = pd.read_csv('liver_cirrhosis_data.csv')
# df = engineer_liver_cirrhosis_features_row_wise(df)
# print(df.head())


In [82]:
augment_data=engineer_liver_cirrhosis_features_row_wise(train_data)

In [96]:
augment_test_data=engineer_liver_cirrhosis_features_row_wise(test_data)

In [97]:
augment_test_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,...,Stage,Clinical_Signs_NA_Count,All_Clinical_Signs_NA,Drug_NA_Clinical_NA_Count,Bilirubin_Status,Ascites_Sex_Interaction,Hepatomegaly_Sex_Interaction,Spiders_Sex_Interaction,Edema_Sex_Interaction,NA_Pattern
0,15000,130.0,D-penicillamine,16944.0,F,Y,Y,Y,N,17.4,...,4.0,0,0,0,High,0,0,0,0,000000
1,15001,2574.0,D-penicillamine,17664.0,F,N,Y,N,N,0.9,...,3.0,0,0,0,Normal,0,0,0,0,000000
2,15002,3853.0,Placebo,13736.0,F,N,N,Y,N,1.6,...,4.0,0,0,0,Normal,0,0,0,0,000000
3,15003,2249.0,NaN,23011.0,F,NaN,NaN,NaN,N,0.9,...,4.0,3,0,3,Normal,0,0,0,0,111000
4,15004,1150.0,NaN,17046.0,F,NaN,NaN,NaN,N,0.7,...,2.0,3,0,3,Normal,0,0,0,0,111000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,2713.0,NaN,17532.0,F,NaN,NaN,NaN,N,1.1,...,3.0,3,0,3,Normal,0,0,0,0,111000
9996,24996,2580.0,D-penicillamine,25569.0,F,N,N,N,N,0.4,...,3.0,0,0,0,Normal,0,0,0,0,000000
9997,24997,186.0,Placebo,21483.0,F,N,Y,Y,S,6.6,...,4.0,0,0,0,Normal,0,0,0,0,000000
9998,24998,2221.0,Placebo,16728.0,F,N,Y,N,N,0.9,...,2.0,0,0,0,Normal,0,0,0,0,000000


In [83]:
augment_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,...,Status,Clinical_Signs_NA_Count,All_Clinical_Signs_NA,Drug_NA_Clinical_NA_Count,Bilirubin_Status,Ascites_Sex_Interaction,Hepatomegaly_Sex_Interaction,Spiders_Sex_Interaction,Edema_Sex_Interaction,NA_Pattern
0,0,3157.0,NaN,24472.0,F,NaN,NaN,NaN,N,1.9,...,C,3,0,3,Normal,0,0,0,0,111000
1,1,1568.0,Placebo,19698.0,F,Y,Y,N,Y,2.5,...,D,0,0,0,Normal,0,0,0,0,000000
2,2,1367.0,NaN,20819.0,F,NaN,NaN,NaN,N,2.0,...,D,3,0,3,Normal,0,0,0,0,111000
3,3,1092.0,NaN,14610.0,F,NaN,NaN,NaN,N,2.9,...,C,3,0,3,Normal,0,0,0,0,111000
4,4,1980.0,NaN,18628.0,F,NaN,NaN,NaN,N,0.5,...,C,3,0,3,Normal,0,0,0,0,111000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,1328.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.7,...,D,3,0,3,Normal,0,0,0,0,111000
14996,14996,904.0,D-penicillamine,22336.0,F,N,N,Y,N,0.6,...,D,0,0,0,Normal,0,0,0,0,000000
14997,14997,989.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.6,...,C,3,0,3,Normal,0,0,0,0,111000
14998,14998,790.0,Placebo,19994.0,F,N,Y,N,N,3.2,...,D,0,0,0,Normal,0,0,0,0,000000


In [62]:
augment_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,...,Bilirubin_Edema_Both_NA,Drug_NA_Clinical_NA_Count,N_Days_NA_Impact,Clinical_Severity_Score,Bilirubin_Status,Ascites_Sex_Interaction,Hepatomegaly_Sex_Interaction,Spiders_Sex_Interaction,Edema_Sex_Interaction,NA_Pattern
0,0,3157.0,NaN,24472.0,F,NaN,NaN,NaN,N,1.9,...,0,3,0,0,Normal,0,0,0,0,111000
1,1,1568.0,Placebo,19698.0,F,Y,Y,N,Y,2.5,...,0,0,0,0,Normal,0,0,0,0,000000
2,2,1367.0,NaN,20819.0,F,NaN,NaN,NaN,N,2.0,...,0,3,0,0,Normal,0,0,0,0,111000
3,3,1092.0,NaN,14610.0,F,NaN,NaN,NaN,N,2.9,...,0,3,0,0,Normal,0,0,0,0,111000
4,4,1980.0,NaN,18628.0,F,NaN,NaN,NaN,N,0.5,...,0,3,0,0,Normal,0,0,0,0,111000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,1328.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.7,...,0,3,0,0,Normal,0,0,0,0,111000
14996,14996,904.0,D-penicillamine,22336.0,F,N,N,Y,N,0.6,...,0,0,0,0,Normal,0,0,0,0,000000
14997,14997,989.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.6,...,0,3,0,0,Normal,0,0,0,0,111000
14998,14998,790.0,Placebo,19994.0,F,N,Y,N,N,3.2,...,0,0,0,0,Normal,0,0,0,0,000000


In [80]:
augment_data['Bilirubin_Status'].unique()

array(['Normal', 'High'], dtype=object)

## pseudo data 이용하는 경우

In [47]:
# #pseudo data
# # pseudo_train_data=np.load('pseudo_train_data.npy',allow_pickle=True)
# # pseudo_train_label=np.load('pseudo_train_label.npy')
# pseudo_train_data=np.load('total_pseudo_data_90per.npy',allow_pickle=True)


# pseudo_train_data=pd.DataFrame(pseudo_train_data)
# pseudo_train_data.columns=['id','N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly','Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper','Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin','Stage', 'Status']
# pseudo_train_label=pseudo_train_data['Status']
# print(pseudo_train_label)
# print(pseudo_train_data.shape)

# le = LabelEncoder()
# pseudo_train_label = le.fit_transform(pseudo_train_label)  
# print(pseudo_train_label)
# pseudo_train_data





## Fill the missing value and add the data for XGboost
데이터의 특성을 고려하여 CNN과 XGBoost에 적용

먼저 categorical과 continuous로 분류

#### Categorical columns:

Drug, Sex, Ascites, Hepatomegaly, Spiders, Edema

#### Continuous columns:

N_Days, Age, Bilirubin, Cholesterol, Albumin, Copper, Alk_Phos, SGOT, Tryglicerides


#### CNN을 위한 처리:


Continuous 변수: 각 컬럼별로 scaling을 적용하고 missing value는 mean으로 채웁니다.
Categorical 변수: one-hot encoding을 적용합니다.


#### XGBoost를 위한 처리:


Continuous 변수: 원래 값을 그대로 사용합니다.
Categorical 변수: 숫자로 인코딩합니다 (예: label encoding).

두 모델 모두에 대해 missing value는 IterativeImputer를 사용하여 처리합니다.
다음은 이를 구현하는 Python 코드 예시입니다:
Data Preprocessing for CNN and XGBoostClick to open code
이 코드는 CNN과 XGBoost에 적합한 형태로 데이터를 전처리합니다:

#### CNN용 처리:

Continuous 변수: SimpleImputer로 평균값 대체 후 StandardScaler로 스케일링
Categorical 변수: SimpleImputer로 'missing' 값 대체 후 OneHotEncoder로 인코딩


#### XGBoost용 처리:

Continuous 변수: 원래 값 유지
Categorical 변수: LabelEncoder로 숫자 인코딩
모든 변수에 대해 IterativeImputer로 결측치 처리



이 방식으로 각 모델의 특성에 맞게 데이터를 준비할 수 있습니다. CNN은 정규화된 수치 입력을, XGBoost는 원래의 연속형 변수와 인코딩된 범주형 변수를 사용하게 됩니다.

In [55]:
# simple imputer
cat_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema','Stage','Clinical_Signs_NA_Count','All_Clinical_Signs_NA','Drug_NA_Clinical_NA_Count','Bilirubin_Status','Ascites_Sex_Interaction','Hepatomegaly_Sex_Interaction','Spiders_Sex_Interaction','Edema_Sex_Interaction','NA_Pattern']
cont_cols = ['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides','Platelets','Prothrombin']

print(len(cat_cols)+len(cont_cols) == 18 , f': if True then Good, total {len(cat_cols)+len(cont_cols)} features ')

# XGBoost를 위한 처리
def preprocess_for_xgboost(df):
    # Continuous 변수는 그대로 사용
    X_cont = df[cont_cols]
    
    # Categorical 변수 인코딩
    X_cat = df[cat_cols].copy()
    for col in cat_cols:
        le = LabelEncoder()
        X_cat[col] = le.fit_transform(X_cat[col].astype(str))
    
    X_xgb = pd.concat([X_cont, X_cat], axis=1)
    
    # Missing value 처리
#     imputer = IterativeImputer()
#     X_xgb_imputed = imputer.fit_transform(X_xgb)
    # Apply SimpleImputer with 'most_frequent' strategy for missing values
    imputer = SimpleImputer(strategy='most_frequent')
    X_xgb_imputed = imputer.fit_transform(X_xgb)
    
    return pd.DataFrame(X_xgb_imputed, columns=X_xgb.columns)


X_simple_xgb = preprocess_for_xgboost(train_data)
X_simple_xgb

True : if True then Good, total 18 features 


,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Stage
0,3157.0,24472.0,1.9,263.0,3.81,20.0,663.0,57.35,85.0,141.0,10.9,2.0,0.0,2.0,2.0,2.0,0.0,3.0
1,1568.0,19698.0,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,12.7,1.0,0.0,1.0,1.0,0.0,2.0,3.0
2,1367.0,20819.0,2.0,263.0,3.05,20.0,663.0,57.35,85.0,80.0,11.3,2.0,0.0,2.0,2.0,2.0,0.0,3.0
3,1092.0,14610.0,2.9,263.0,3.73,20.0,663.0,57.35,85.0,337.0,10.2,2.0,0.0,2.0,2.0,2.0,0.0,3.0
4,1980.0,18628.0,0.5,263.0,3.12,20.0,663.0,57.35,85.0,190.0,11.2,2.0,0.0,2.0,2.0,2.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1328.0,23376.0,0.7,263.0,3.14,20.0,663.0,57.35,85.0,425.0,10.9,2.0,0.0,2.0,2.0,2.0,0.0,2.0
14996,904.0,22336.0,0.6,396.0,3.53,102.0,1257.0,137.95,118.0,216.0,10.6,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14997,989.0,23376.0,0.6,263.0,3.40,20.0,663.0,57.35,85.0,388.0,11.0,2.0,0.0,2.0,2.0,2.0,0.0,2.0
14998,790.0,19994.0,3.2,263.0,3.41,86.0,1790.0,134.85,85.0,149.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0


In [86]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

# Define categorical and continuous columns
# cat_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema','Stage']
# cont_cols = ['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides','Platelets','Prothrombin']
cat_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema','Stage','Clinical_Signs_NA_Count','All_Clinical_Signs_NA','Drug_NA_Clinical_NA_Count','Bilirubin_Status','Ascites_Sex_Interaction','Hepatomegaly_Sex_Interaction','Spiders_Sex_Interaction','Edema_Sex_Interaction','NA_Pattern']
cont_cols = ['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides','Platelets','Prothrombin']

print(len(cat_cols) + len(cont_cols) == 18, f': if True then Good, total {len(cat_cols) + len(cont_cols)} features')

# Preprocessing function for XGBoost
def random_preprocess_for_xgboost(df):
    # Continuous variables handling
    X_cont = df[cont_cols].copy()
    for feat in cont_cols:
        non_nan_values = X_cont[feat].dropna().values
        X_cont[feat] = X_cont[feat].apply(lambda x: np.random.choice(non_nan_values) if pd.isna(x) else x)
    
    # Categorical variables handling
    X_cat = df[cat_cols].copy()
    for col in cat_cols:
        le = LabelEncoder()
        # Fit the label encoder
        X_cat[col] = le.fit_transform(X_cat[col].astype(str))
        # Add a new label for missing values (next integer value)
        max_label = X_cat[col].max() + 1
        X_cat[col] = X_cat[col].fillna(max_label)
    
    # Combine continuous and categorical variables
    X_xgb = pd.concat([X_cont, X_cat], axis=1)
    
    return X_xgb

# Apply preprocessing to the train_data
# X_random_xgb = random_preprocess_for_xgboost(train_data)
X_random_xgb = random_preprocess_for_xgboost(augment_data)

False : if True then Good, total 27 features


In [25]:
X_random_xgb

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Stage
0,3157.0,24472.0,1.9,426.0,3.81,57.0,855.0,71.30,242.0,141.0,10.9,2,0,2,2,2,0,3
1,1568.0,19698.0,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,12.7,1,0,1,1,0,2,3
2,1367.0,20819.0,2.0,198.0,3.05,77.0,9066.8,134.85,189.0,80.0,11.3,2,0,2,2,2,0,3
3,1092.0,14610.0,2.9,250.0,3.73,73.0,794.0,60.45,111.0,337.0,10.2,2,0,2,2,2,0,3
4,1980.0,18628.0,0.5,232.0,3.12,70.0,663.0,55.80,165.0,190.0,11.2,2,0,2,2,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1328.0,23376.0,0.7,235.0,3.14,58.0,1768.0,57.35,58.0,425.0,10.9,2,0,2,2,2,0,2
14996,904.0,22336.0,0.6,396.0,3.53,102.0,1257.0,137.95,118.0,216.0,10.6,0,0,0,0,1,0,0
14997,989.0,23376.0,0.6,288.0,3.40,52.0,1052.0,55.80,77.0,388.0,11.0,2,0,2,2,2,0,2
14998,790.0,19994.0,3.2,578.0,3.41,86.0,1790.0,134.85,189.0,149.0,11.0,1,0,0,1,0,0,3


In [87]:
X_random_xgb

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,...,Stage,Clinical_Signs_NA_Count,All_Clinical_Signs_NA,Drug_NA_Clinical_NA_Count,Bilirubin_Status,Ascites_Sex_Interaction,Hepatomegaly_Sex_Interaction,Spiders_Sex_Interaction,Edema_Sex_Interaction,NA_Pattern
0,3157.0,24472.0,1.9,450.0,3.81,52.0,2184.0,46.50,107.0,141.0,...,3,3,0,3,1,0,0,0,0,7
1,1568.0,19698.0,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,...,3,0,0,0,1,0,0,0,0,0
2,1367.0,20819.0,2.0,244.0,3.05,13.0,884.0,56.76,68.0,80.0,...,3,3,0,3,1,0,0,0,0,7
3,1092.0,14610.0,2.9,263.0,3.73,159.0,688.0,97.65,91.0,337.0,...,3,3,0,3,1,0,0,0,0,7
4,1980.0,18628.0,0.5,420.0,3.12,200.0,2310.0,161.20,113.0,190.0,...,3,3,0,3,1,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1328.0,23376.0,0.7,400.0,3.14,28.0,897.0,75.95,91.0,425.0,...,2,3,0,3,1,0,0,0,0,7
14996,904.0,22336.0,0.6,396.0,3.53,102.0,1257.0,137.95,118.0,216.0,...,0,0,0,0,1,0,0,0,0,0
14997,989.0,23376.0,0.6,324.0,3.40,74.0,1553.0,57.35,85.0,388.0,...,2,3,0,3,1,0,0,0,0,7
14998,790.0,19994.0,3.2,268.0,3.41,86.0,1790.0,134.85,96.0,149.0,...,3,0,0,0,1,0,0,0,0,0


In [92]:
le = LabelEncoder()
train_label = le.fit_transform(train_label)  
print(train_label)

[0 2 2 ... 0 2 0]


## pseudo train data, label preprocessing

In [9]:
X_pseudo_xgb=random_preprocess_for_xgboost(pseudo_train_data)
X_pseudo_xgb

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Drug,Sex,Ascites,Hepatomegaly,Spiders,Edema,Stage
0,3157.0,24472.0,1.9,345.0,3.81,46.0,790.0,66.65,114.0,141.0,10.9,2,0,2,2,2,0,3
1,1568.0,19698.0,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,12.7,1,0,1,1,0,2,3
2,1367.0,20819.0,2.0,252.0,3.05,96.0,1134.0,196.85,128.0,80.0,11.3,2,0,2,2,2,0,3
3,1092.0,14610.0,2.9,486.0,3.73,38.0,648.0,116.25,168.0,337.0,10.2,2,0,2,2,2,0,3
4,1980.0,18628.0,0.5,248.0,3.12,24.0,976.0,57.35,107.0,190.0,11.2,2,0,2,2,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20530,2149.0,22336.0,0.6,248.0,3.89,20.0,678.0,58.00,84.0,128.0,10.0,0,0,0,0,0,0,1
20531,1433.0,20510.0,0.7,298.0,4.01,28.0,733.0,65.10,58.0,256.0,10.7,1,0,0,0,0,0,1
20532,2580.0,25569.0,0.4,315.0,4.01,20.0,666.0,54.25,242.0,277.0,10.0,0,0,0,0,0,0,2
20533,186.0,21483.0,6.6,1000.0,3.50,188.0,944.0,130.20,133.0,265.0,11.0,1,0,0,1,1,1,3


In [10]:
print(pseudo_train_label.shape)
pseudo_train_label

(20535,)


array([0, 2, 2, ..., 0, 2, 2])

In [19]:
print(X_simple_xgb.shape,X_random_xgb.shape,X_pseudo_xgb.shape)

(20535, 18)


## 성능 비교 SIMPLEIMPUTER VS ITERATIVE IMPUTER(~MISSINGFOREST)

In [90]:
#data
# print(X_simple_xgb.shape,X_random_xgb.shape,X_pseudo_xgb.shape)

X_train=np.array(X_random_xgb) #랜덤이 가장 잘 뽑힌다고 함!
train_label=np.array(train_label) #pseudo_train_label

#pseudo version
# X_train=np.array(X_pseudo_xgb) #랜덤이 가장 잘h 뽑힌다고 !
# train_label=np.array(pseudo_train_label) #pseudo_train_label

In [12]:
#Cross validation 할때만 사용, Grid search는 CPU만 가능!

# train_= torch.tensor(X_simple_xgb,dtype=torch.float64)
# label_= torch.tensor(train_label,dtype=torch.uint8)


## 직렬 sequential random search이용

In [149]:
from sklearn.model_selection import RandomizedSearchCV



def find_best_xgb_parameter(X_train, y_train, model):
    # First step: Tuning 'max_depth' and 'min_child_weight'
    params = {
        'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
        'min_child_weight': [1, 2, 3, 5, 7, 10, 15]
    }
    xgb_cv = RandomizedSearchCV(model, params, cv=5, n_jobs=-1, n_iter=56, random_state=42, scoring='neg_log_loss')
    xgb_cv.fit(X_train, y_train)
    print('stage 1 clear')

    # Second step: Tuning 'subsample' and 'colsample_bytree'
    params = {
        'subsample': [0.05, 0.1, 0.25, 0.5, 0.75, 0.9,  1.0],
        'colsample_bytree': [0.05, 0.1, 0.25, 0.5, 0.75, 1.0]
    }
    model = xgb_cv.best_estimator_
    xgb_cv = RandomizedSearchCV(model, params, cv=5, n_jobs=-1, n_iter=42, random_state=42, scoring='neg_log_loss')
    xgb_cv.fit(X_train, y_train)
    print('stage 2 clear')

    # Third step: Tuning 'learning_rate' and 'n_estimators'
    params = {
        'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
        'n_estimators': [100, 150, 200, 250, 300]
    }
    model = xgb_cv.best_estimator_
    xgb_cv = RandomizedSearchCV(model, params, cv=3, n_jobs=-1, n_iter=40, random_state=42, scoring='neg_log_loss')
    xgb_cv.fit(X_train, y_train)
    
    print('stage 3 clear')
    
    # Fourth step: Tuning 'gamma', 'reg_lambda', and 'reg_alpha'
    params = {
        'gamma': [0,0.01,0.1,0.5, 1],
        'reg_lambda': [0, 0.01, 0.1, 0.5, 1],
        'reg_alpha': [0,0.01, 0.1, 0.5, 1]
    }
    model = xgb_cv.best_estimator_
    xgb_cv = RandomizedSearchCV(model, params, cv=3, n_jobs=-1, n_iter=100, random_state=42, scoring='neg_log_loss')
    xgb_cv.fit(X_train, y_train)
    
    print('stage 4 clear - end')

    return xgb_cv


In [27]:

# Initialize the XGBoost model
model = xgb.XGBClassifier(tree_method='hist', eval_metric='mlogloss', device='cuda', verbose=2)

# Call the sequential tuning function
# best_model_cv = find_best_xgb_parameter(X_train,train_label, model)
# best_model_cv = find_best_xgb_parameter_ver2(X_train,train_label)
best_model_cv=find_better_xgb_parameter(X_train,train_label)

# After running the tuning function, the `best_model_cv` variable will contain the best estimator
# You can then use this tuned model for predictions or further evaluation
print("Best hyperparameters:", best_model_cv.best_params_)
print("Best score:", best_model_cv.best_score_) 
#1회: 3.7  2회: -0.37186824607626434  3회(pseudo):  -0.2860589699142558 (but peform worse on test data)
# 4회(simple) : bad #5회, random search parallel:(random) Best score: -0.3657089375934389(best) #

#random parallel search
# Best hyperparameters: {'subsample': 0.9, 'reg_lambda': 0.01, 'reg_alpha': 0.5, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.25}
# Best score: -0.3657089375934389

Fitting 5 folds for each of 150 candidates, totalling 750 fits


KeyboardInterrupt: 

In [150]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

def find_best_xgb_parameter_ver2(X_train, y_train):
    # Define the complete parameter grid for simultaneous tuning
    param_grid = {
        'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
        'min_child_weight': [1, 2, 3, 5, 7, 10, 15],
        'subsample': [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0],
        'colsample_bytree': [0.01,0.05, 0.1, 0.25, 0.5, 0.75, 1.0],
        'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
        'n_estimators': [100, 150, 200, 250, 300,400,500,700],
        'gamma': [0, 0.01, 0.1, 0.5, 1],
        'reg_lambda': [0, 0.01, 0.1, 0.5, 1],
        'reg_alpha': [0, 0.01, 0.1, 0.5, 1]
    }

    # Initialize the XGBoost model with GPU support
    model = xgb.XGBClassifier(
        tree_method='hist',  # Use GPU for faster training
        use_label_encoder=False,
        device='cuda',
        eval_metric='mlogloss',  # Log loss for multi-class classification
        objective='multi:softprob',
        num_class=3  # Adjust this to the number of classes in your dataset
    )

    # Perform RandomizedSearchCV over the entire parameter space
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=200,  # Number of random parameter combinations to try
        scoring='neg_log_loss',
        n_jobs=-1,
        cv=5,  # 5-fold cross-validation
        verbose=2,
        random_state=44
    )

    # Fit the model with RandomizedSearchCV
    random_search.fit(X_train, y_train)

    # Return the best estimator and its parameters
    return random_search


In [23]:
def find_better_xgb_parameter(X_train, y_train):
    # Refined parameter grid for improved performance
    param_grid = {
        'max_depth': [8, 9, 10, 11, 12],  # Fine-tuning around best value (10)
        'min_child_weight': [4, 5, 6],  # Fine-tuning around best value (5)
        'subsample': [0.8, 0.85, 0.9, 0.95],  # Fine-tuning around 0.9
        'colsample_bytree': [0.2, 0.25, 0.3],  # Fine-tuning around 0.25
        'learning_rate': [0.03, 0.05, 0.07],  # Exploring around 0.05
        'n_estimators': [180, 200, 220, 250],  # More estimators with lower learning rate
        'gamma': [0.05, 0.1, 0.15],  # Fine-tuning around 0.1
        'reg_lambda': [0.005, 0.01, 0.02],  # Fine-tuning around 0.01
        'reg_alpha': [0.4, 0.5, 0.6]  # Fine-tuning around 0.5
    }
# 
    # Initialize the XGBoost model with GPU support
    model = xgb.XGBClassifier(
        tree_method='hist',  # Use GPU for faster training
        use_label_encoder=False,
        device='cuda',
        eval_metric='mlogloss',
        objective='multi:softprob',
        num_class=3  # Adjust this to the number of classes in your dataset
    )

    # Perform RandomizedSearchCV over the refined parameter space
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=150,  # Fewer iterations due to more focused grid
        scoring='neg_log_loss',
        n_jobs=-1,
        cv=5,  # 5-fold cross-validation
        verbose=2,
        random_state=44
    )

    # Fit the model with RandomizedSearchCV
    random_search.fit(X_train, y_train)

    # Return the best estimator and its parameters
    return random_search


In [29]:
# !conda install optuna

^C


In [94]:
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial):
#     params = {
#         'objective': 'multi:softprob',
#         'num_class': 3,
#         'max_depth': trial.suggest_int('max_depth', 5, 25),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
#         'subsample': trial.suggest_float('subsample', 0.01, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 700),
#         'random_state': 2
#     }
    
    
    params = {
        'objective': 'multi:softprob',
        'num_class': 3,
        'max_depth': trial.suggest_int('max_depth', 10, 20),  # Increased to fine-tune around 15
        'min_child_weight': trial.suggest_int('min_child_weight', 8, 16),  # Narrowed around the best value (10)
        'subsample': trial.suggest_float('subsample', 0.95, 1.0),  # Focused on high subsample values near 0.997
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.2),  # Focused around 0.153
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.04),  # Narrowed around 0.023
        'n_estimators': trial.suggest_int('n_estimators', 500, 700),  # Fine-tuning around 611
        'random_state': 2
    }

    model = xgb.XGBClassifier(**params, verbosity=0, device='cuda', n_jobs=-1)
    scores = cross_val_score(model, X_train, train_label, cv=5, scoring='neg_log_loss')
    return -scores.mean()

    
    model = xgb.XGBClassifier(**params, verbosity=0, device='cuda', n_jobs=-1)
    scores = cross_val_score(model, X_train, train_label, cv=5, scoring='neg_log_loss')
    return -scores.mean()

def optimize_xgboost(X_train, y_train, n_trials=250):
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    return study.best_params

def train_model(X_train, y_train, best_params):
    model = xgb.XGBClassifier(**best_params, device='cuda', n_jobs=-1)
    model.fit(X_train, y_train)
    return model

# Assuming X_train and y_train are already defined
best_params = optimize_xgboost(X_train, train_label)
final_model = train_model(X_train,train_label, best_params)

# You can now use final_model for predictions
# predictions = final_model.predict(X_test)

[I 2024-10-17 23:18:46,729] A new study created in memory with name: no-name-d6457830-5c13-453e-b6c1-9a2835398921
[I 2024-10-17 23:19:30,185] Trial 0 finished with value: 0.3726835747437166 and parameters: {'max_depth': 12, 'min_child_weight': 9, 'subsample': 0.9908282608007444, 'colsample_bytree': 0.14190781387612939, 'learning_rate': 0.01960593318844768, 'n_estimators': 509}. Best is trial 0 with value: 0.3726835747437166.
[I 2024-10-17 23:20:24,993] Trial 1 finished with value: 0.3636967653195471 and parameters: {'max_depth': 20, 'min_child_weight': 14, 'subsample': 0.9796593436843652, 'colsample_bytree': 0.17672867852921462, 'learning_rate': 0.031173727093095988, 'n_estimators': 607}. Best is trial 1 with value: 0.3636967653195471.
[I 2024-10-17 23:21:17,538] Trial 2 finished with value: 0.36185839693914873 and parameters: {'max_depth': 12, 'min_child_weight': 8, 'subsample': 0.9665601547562184, 'colsample_bytree': 0.12355113792160276, 'learning_rate': 0.0262390732061374, 'n_estima

[W 2024-10-17 23:33:47,878] Trial 17 failed with value None.


KeyboardInterrupt: 

In [100]:
# ##병렬적인 random search --> 효율  bad
# from sklearn.model_selection import RandomizedSearchCV


# # 데이터 준비
# X_train, X_val, y_train, y_val = train_test_split(X_train, train_label, test_size=0.2, random_state=42)

# # XGBoost의 DMatrix 형식으로 변환
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dval = xgb.DMatrix(X_val, label=y_val)

# # 파라미터 그리드 정의 (범위를 좁힘)

# do sequentially, 

# param_grid = {
#     'max_depth': [3, 4,5,6,7,8,10,12,15],
#     'learning_rate': [0.001,0.05, 0.1,0.15,0.2,0.25,0.3],
#     'n_estimators': [100, 150, 200, 250,300],
#     'subsample': [0.05 , 0.1,0.25 , 0.5 , 0.75, 0.9, 1.0],
#     'colsample_bytree': [0.05,0.1,0.25,0.5,0.75,1.0],
#     'gamma': [0, 1],
#     'min_child_weight': [1, 2, 3, 5, 6 ,10,15], 
#     'reg_lambda': [0,0.1, 0.5,1],
#     'reg_alpha': [0, 0.1, 0.5, 1]
# }

# # 기본 파라미터 설정
# base_params = {
#     'objective': 'multi:softprob',
#     'num_class': 3,
#     'tree_method': 'hist',
#     'device': 'cuda',
#     'eval_metric': 'mlogloss'
# }

# def objective(params):
#     # 기본 파라미터와 탐색 파라미터 결합
#     params.update(base_params)
    
#     # 학습
#     num_round = params.pop('n_estimators')
#     model = xgb.train(
#         params,
#         dtrain,
#         num_round,
#         evals=[(dval, 'eval')],
#         early_stopping_rounds=10,
#         verbose_eval=False
#     )
    
#     # 검증 세트에 대한 로그 손실 반환
#     return model.best_score

# # RandomizedSearchCV 사용
# random_search = RandomizedSearchCV(
#     estimator=xgb.XGBClassifier(tree_method='hist', device='cuda'),
#     param_distributions=param_grid,
#     n_iter=50,  # 100번의 랜덤 조합 시도
#     scoring='neg_log_loss',
#     n_jobs=-1,  # XGBoost가 내부적으로 병렬 처리를 하므로 1로 설정
#     cv=5,
#     verbose=2,
#     random_state=42
# )

# # 학습 실행
# random_search.fit(X_train, y_train)

# print("Best hyperparameters:", random_search.best_params_)
# print("Best score:", -random_search.best_score_)

# # Best hyperparameters: {'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 0.8}
# # Best score: 0.3736724319627851

In [174]:
import pickle

# # Save the best params to a pickle file
with open("final_best_params.pkl", "wb") as file:
#     pickle.dump(grid_search.best_params_, file)
    pickle.dump(best_model_cv.best_params_, file)
    
    


# Load the best params from the pickle file
with open("final_best_params.pkl", "rb") as file:
    best_params = pickle.load(file)

# You can now use the loaded parameters
print(best_params)


{'subsample': 0.9, 'reg_lambda': 0.01, 'reg_alpha': 0.5, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.25}


## Result for test data 제출


In [98]:
#best model 로 제출하기 log loss for test data 계산?

#test data preprocessing

# simple imputer
print(test_data['id'])

# cat_cols = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema','Stage']
# cont_cols = ['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides','Platelets','Prothrombin']


# X_test_random_xgb=random_preprocess_for_xgboost(test_data)
X_test_random_xgb=random_preprocess_for_xgboost(augment_test_data)
X_test_random_xgb 

# X_test_simple_xgb=preprocess_for_xgboost(test_data)
# X_test_simple_xgb 


0       15000
1       15001
2       15002
3       15003
4       15004
        ...  
9995    24995
9996    24996
9997    24997
9998    24998
9999    24999
Name: id, Length: 10000, dtype: int64


,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,...,Stage,Clinical_Signs_NA_Count,All_Clinical_Signs_NA,Drug_NA_Clinical_NA_Count,Bilirubin_Status,Ascites_Sex_Interaction,Hepatomegaly_Sex_Interaction,Spiders_Sex_Interaction,Edema_Sex_Interaction,NA_Pattern
0,130.0,16944.0,17.4,299.0,3.00,182.0,559.0,119.35,73.0,401.0,...,3,0,0,0,0,0,0,0,0,0
1,2574.0,17664.0,0.9,242.0,3.65,108.0,1040.0,108.50,118.0,344.0,...,2,0,0,0,1,0,0,0,0,0
2,3853.0,13736.0,1.6,354.0,3.80,44.0,1584.0,111.60,108.0,277.0,...,3,0,0,0,1,0,0,0,0,0
3,2249.0,23011.0,0.9,242.0,3.06,33.0,784.0,120.90,46.0,190.0,...,3,3,0,2,1,0,0,0,0,6
4,1150.0,17046.0,0.7,404.0,3.66,29.0,784.0,98.00,85.0,350.0,...,1,3,0,2,1,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2713.0,17532.0,1.1,252.0,3.75,57.0,369.0,147.25,598.0,330.0,...,2,3,0,2,1,0,0,0,0,6
9996,2580.0,25569.0,0.4,223.0,4.01,20.0,666.0,54.25,177.0,277.0,...,2,0,0,0,1,0,0,0,0,0
9997,186.0,21483.0,6.6,1000.0,3.50,188.0,944.0,130.20,133.0,265.0,...,3,0,0,0,1,0,0,0,0,0
9998,2221.0,16728.0,0.9,434.0,3.36,161.0,1523.0,117.80,166.0,381.0,...,1,0,0,0,1,0,0,0,0,0


In [36]:

# Assuming you have already loaded your test dataset as 'X_test' and trained model as 'xgb_model'
# Predict probabilities for each class
# xgb_probs = best_model_cv.predict_proba(X_test_random_xgb)
# xgb_probs = best_model_cv.predict_proba(X_test_random_xgb)
# xgb_probs = final_model.predict_proba(X_test_random_xgb)

test_id = test_data['id']


In [154]:
output_df = pd.DataFrame(xgb_probs, columns=['Status_C', 'Status_CL', 'Status_D'])
## if only using XGB model?
# output_df = pd.DataFrame(xgb_probs, columns=['Status_C', 'Status_CL', 'Status_D'])
output_df.insert(0, 'id', test_id)

# Save to a CSV file
output_df.to_csv('Tony_Final_submission_random_no_pseudo_ver2.csv', index=False)


In [158]:

train_label

array([0, 2, 2, ..., 0, 2, 0])

(15000,)

## using multiple XGB for averaging:



In [102]:
# Number of models to train
import xgboost as xgb

# Number of models to train
num_models = 5
models = []

X_pseudo_simple_xgb=X_train
pseudo_train_label=train_label

#final_param={'max_depth': 16, 'min_child_weight': 14, 'subsample': 0.9237806795685649, 'colsample_bytree': 0.14309918521189277, 'learning_rate': 0.03883023984291649, 'n_estimators': 522}
# final_param={'max_depth': 15, 'min_child_weight': 10, 'subsample': 0.9974547490148263, 'colsample_bytree': 0.15312805153329764, 'learning_rate': 0.023214911136020238, 'n_estimators': 611}
final_param={'max_depth': 12, 'min_child_weight': 8, 'subsample': 0.9665601547562184, 'colsample_bytree': 0.12355113792160276, 'learning_rate': 0.0262390732061374, 'n_estimators': 687}
# Initialize an array to store the predictions
test_probs = np.zeros((X_test_random_xgb.shape[0], 3))  # Assuming 3 classes

for i in range(num_models):
    # Initialize the model with best parameters and changing random seed
    model = xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=3,
        tree_method='hist',
        device='cuda',
        eval_metric='mlogloss',
        **final_param,
        random_state=i
    )
    
    
    # Train the model
    model.fit(X_pseudo_simple_xgb, pseudo_train_label)

    # Store the model in the list
    models.append(model)

for model in models:
    # Get probabilities from each model and accumulate them
    
    test_probs += model.predict_proba(X_test_random_xgb)
    print(test_probs)

# Average the accumulated probabilities
test_probs /= num_models

output_df = pd.DataFrame(test_probs, columns=['Status_C', 'Status_CL', 'Status_D'])
output_df.insert(0, 'id', test_id)  # Assuming test_id contains the IDs for your test data
output_df.to_csv('Tony_final_ensemble_ver5_with_augment.csv', index=False)


[[0.00280889 0.00265506 0.99453604]
 [0.7140094  0.06815347 0.2178371 ]
 [0.92922771 0.01105426 0.05971797]
 ...
 [0.01945707 0.00413852 0.97640443]
 [0.74147075 0.03380461 0.22472465]
 [0.00759213 0.00132308 0.99108475]]
[[0.00738404 0.00632471 1.98629129]
 [1.41491812 0.13747316 0.44760868]
 [1.87378877 0.02070533 0.10550585]
 ...
 [0.04193143 0.00820617 1.94986242]
 [1.46922028 0.06222029 0.46855941]
 [0.01764253 0.00265476 1.97970271]]
[[0.01035154 0.00948547 2.98016298]
 [2.11893177 0.20022728 0.68084086]
 [2.80066919 0.03555001 0.16378075]
 ...
 [0.06554406 0.01319144 2.92126453]
 [2.19477159 0.08662806 0.71860033]
 [0.02751244 0.0039944  2.9684931 ]]
[[0.01281099 0.01220345 3.9749856 ]
 [2.88193405 0.2549128  0.8631531 ]
 [3.73634511 0.04959954 0.21405533]
 ...
 [0.08529862 0.01784809 3.89685333]
 [2.95563018 0.11620109 0.92816874]
 [0.03504108 0.00531814 3.95964074]]
[[0.01576139 0.01611171 4.96812695]
 [3.60494173 0.3174906  1.07756761]
 [4.66225201 0.06333373 0.27441429]
 ...

In [37]:

output_df = pd.DataFrame(test_probs, columns=['Status_C', 'Status_CL', 'Status_D'])
output_df.insert(0, 'id', test_id)  # Assuming test_id contains the IDs for your test data
output_df.to_csv('Tony_final_ensemble_ver4.csv', index=False)


## Pseudo data 만들기 !!

In [2]:
#make pseudo labels:

confident_ids=[]
predicted= pd.read_csv('submission_xgb_ensemble_ver2.csv')
# predicted= pd.read_csv('submission.csv')
print(predicted.columns[1:])

confidence_threshold = 0.8
# confident_indices = np.max(predicted[predicted.columns[1:]], axis=1) > confidence_threshold
confident_indices = predicted[predicted.columns[1:]].max(axis=1) > confidence_threshold

confident_ids = predicted.loc[confident_indices, 'id'].tolist()

# Create pseudo labels by finding the column (class) with the highest probability for each confident ID
pseudo_labels = predicted.loc[confident_indices, predicted.columns[1:]].idxmax(axis=1)

# Combine the IDs and their corresponding pseudo labels into a new DataFrame
pseudo_label_df = pd.DataFrame({
    'id': predicted.loc[confident_indices, 'id'],
    'pseudo_label': pseudo_labels
})

# Display or use the pseudo-label DataFrame
print(pseudo_label_df)

confident_ids
print(len(confident_ids))

NameError: name 'pd' is not defined

In [17]:

pseudo_test_data.loc[:, "Status"] = pseudo_labels
pseudo_test_data


,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,15000,130.0,D-penicillamine,16944.0,F,Y,Y,Y,N,17.4,NaN,3.00,182.0,559.0,119.35,NaN,401.0,11.0,4.0,Status_D
2,15002,3853.0,Placebo,13736.0,F,N,N,Y,N,1.6,354.0,3.80,44.0,1584.0,111.60,108.0,277.0,10.3,4.0,Status_C
4,15004,1150.0,NaN,17046.0,F,NaN,NaN,NaN,N,0.7,NaN,3.66,NaN,NaN,NaN,NaN,350.0,10.3,2.0,Status_C
5,15005,2576.0,Placebo,16122.0,F,N,Y,N,N,0.5,256.0,3.87,14.0,423.0,49.60,84.0,336.0,10.4,3.0,Status_C
6,15006,2863.0,D-penicillamine,20736.0,F,N,N,N,N,0.6,251.0,4.38,12.0,674.0,57.00,85.0,344.0,9.9,3.0,Status_C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,24991,2149.0,D-penicillamine,22336.0,F,N,N,N,N,0.6,NaN,3.89,20.0,678.0,58.00,NaN,128.0,10.0,2.0,Status_C
9992,24992,1433.0,Placebo,20510.0,F,N,N,N,N,0.7,298.0,4.01,28.0,733.0,65.10,58.0,256.0,10.7,2.0,Status_C
9996,24996,2580.0,D-penicillamine,25569.0,F,N,N,N,N,0.4,NaN,4.01,20.0,666.0,54.25,NaN,277.0,10.0,3.0,Status_C
9997,24997,186.0,Placebo,21483.0,F,N,Y,Y,S,6.6,1000.0,3.50,188.0,944.0,130.20,133.0,265.0,11.0,4.0,Status_D


In [18]:
test_data

pseudo_test_data = test_data[test_data['id'].isin(confident_ids)]
pseudo_test_data

pseudo_test

NameError: name 'pseudo_test' is not defined

In [1]:
pseudo_labels

# le = LabelEncoder()
# pseudo_label = le.fit_transform(pseudo_labels)  # 'C' -> 0, 'D' -> 1, 'CL' -> 2

# pseudo_label



NameError: name 'pseudo_labels' is not defined

In [139]:
train_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,3157.0,NaN,24472.0,F,NaN,NaN,NaN,N,1.9,NaN,3.81,NaN,NaN,NaN,NaN,141.0,10.9,4.0,C
1,1,1568.0,Placebo,19698.0,F,Y,Y,N,Y,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,12.7,4.0,D
2,2,1367.0,NaN,20819.0,F,NaN,NaN,NaN,N,2.0,NaN,3.05,NaN,NaN,NaN,NaN,80.0,11.3,4.0,D
3,3,1092.0,NaN,14610.0,F,NaN,NaN,NaN,N,2.9,NaN,3.73,NaN,NaN,NaN,NaN,337.0,10.2,4.0,C
4,4,1980.0,NaN,18628.0,F,NaN,NaN,NaN,N,0.5,NaN,3.12,NaN,NaN,NaN,NaN,190.0,11.2,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,1328.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.7,NaN,3.14,NaN,NaN,NaN,NaN,425.0,10.9,3.0,D
14996,14996,904.0,D-penicillamine,22336.0,F,N,N,Y,N,0.6,396.0,3.53,102.0,1257.0,137.95,118.0,216.0,10.6,1.0,D
14997,14997,989.0,NaN,23376.0,F,NaN,NaN,NaN,N,0.6,NaN,3.40,NaN,NaN,NaN,NaN,388.0,11.0,3.0,C
14998,14998,790.0,Placebo,19994.0,F,N,Y,N,N,3.2,NaN,3.41,86.0,1790.0,134.85,NaN,149.0,11.0,4.0,D


In [140]:
#augument training data with pseudo labels+ dataset

pseudo_train_data = pd.concat([train_data, pseudo_test_data], ignore_index=True)
pseudo_train_data

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,3157.0,NaN,24472.0,F,NaN,NaN,NaN,N,1.9,NaN,3.81,NaN,NaN,NaN,NaN,141.0,10.9,4.0,C
1,1,1568.0,Placebo,19698.0,F,Y,Y,N,Y,2.5,178.0,2.56,209.0,815.0,159.65,78.0,149.0,12.7,4.0,D
2,2,1367.0,NaN,20819.0,F,NaN,NaN,NaN,N,2.0,NaN,3.05,NaN,NaN,NaN,NaN,80.0,11.3,4.0,D
3,3,1092.0,NaN,14610.0,F,NaN,NaN,NaN,N,2.9,NaN,3.73,NaN,NaN,NaN,NaN,337.0,10.2,4.0,C
4,4,1980.0,NaN,18628.0,F,NaN,NaN,NaN,N,0.5,NaN,3.12,NaN,NaN,NaN,NaN,190.0,11.2,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22004,24991,2149.0,D-penicillamine,22336.0,F,N,N,N,N,0.6,NaN,3.89,20.0,678.0,58.00,NaN,128.0,10.0,2.0,Status_C
22005,24992,1433.0,Placebo,20510.0,F,N,N,N,N,0.7,298.0,4.01,28.0,733.0,65.10,58.0,256.0,10.7,2.0,Status_C
22006,24996,2580.0,D-penicillamine,25569.0,F,N,N,N,N,0.4,NaN,4.01,20.0,666.0,54.25,NaN,277.0,10.0,3.0,Status_C
22007,24997,186.0,Placebo,21483.0,F,N,Y,Y,S,6.6,1000.0,3.50,188.0,944.0,130.20,133.0,265.0,11.0,4.0,Status_D
